### old das

In [1]:
import numpy as np
from EuCAP2024.umbms.tdelay import propspeed
import pickle
from EuCAP2024.umbms.recon import extras
from EuCAP2024.umbms.tdelay import propspeed
from EuCAP2024.umbms.hardware import antenna as an
from EuCAP2024.umbms.recon import algos
from EuCAP2024.umbms.plot.imgs import plot_img

In [ ]:
roi_radius = 8
m_size = int(roi_radius * 2/0.1)
adi_rad = 5 #!!!!!!!!!!!!!!!!!!!!
antenna_r = 22 + 2.4 #!!!!!!!!!!!!1
frequencies_1001 = np.linspace(1e9, 8e9,1001)
frequencies_288 = frequencies_1001[139::3]

speed = propspeed.estimate_speed(adi_rad, antenna_r)
pix_ts = extras.get_pix_ts(antenna_r, m_size, roi_radius, speed, 72, -136)
a_td = an.apply_ant_t_delay(pix_ts)
pf = extras.get_fd_phase_factor(a_td)

In [ ]:
def load(file_path):
    with open(file_path, 'rb') as f:
        return pickle.load(f)
dataset_wos_original = load("fd_data_s11_adi.pickle")

wos_original = dataset_wos_original[9] #!!!!!!!!!!!!!
wos_original_reducedfq = wos_original[139::3,:]

wos_predicted = np.load("sample_30.npy") #these have to be generated from the skin-substraction ML code
wos_predicted_ = wos_predicted[:,:,0]

In [5]:
das_original = algos.fd_das(np.abs(wos_original), pf, frequencies_1001)
print('--')
das_original_reducedfq = algos.fd_das(np.abs(wos_original_reducedfq), pf, frequencies_288)
print('--')
das_predicted = algos.fd_das(np.abs(wos_predicted_), pf, frequencies_288)

--
--


In [ ]:
mse = np.mean((np.abs(das_original_reducedfq) - np.abs(das_predicted))**2)
print("MSE:", mse)

from skimage.metrics import structural_similarity as ssim
ssim_val, _ = ssim(np.abs(das_original_reducedfq), np.abs(das_predicted), full=True, data_range=np.abs(das_original_reducedfq).max() - np.abs(das_predicted).min())
print("SSIM:", ssim_val)

das_orig_abs = np.abs(das_original_reducedfq) / np.max(np.abs(das_original_reducedfq))
das_pred_abs = np.abs(das_predicted) / np.max(np.abs(das_predicted))

mse_val = np.mean((das_orig_abs - das_pred_abs) ** 2)
ssim_val = ssim(das_orig_abs, das_pred_abs, data_range=1.0)

print(mse_val)
print(ssim_val)

print('Original_1001')
plot_img(img=np.abs(das_original), roi_rho=roi_radius)#, save_fig=True, save_str=os.path.join(r'C:\Users\Gabriela\Desktop', "wv"))
# print('Original_288')
plot_img(img=np.abs(das_original_reducedfq), roi_rho=roi_radius)
# print('Predicted_288')
plot_img(img=np.abs(das_predicted), roi_rho=roi_radius)


MSE: 1.2213722045356392
SSIM: 0.5224219606620119
0.013070479019059835
0.46860123571191836


### Brooke's das

In [ ]:
import os
import time

import numpy as np
import multiprocessing as mp
import matplotlib

matplotlib.use('agg')
import matplotlib.pyplot as plt


from EuCAP2024.umbms import get_proj_path, verify_path, get_script_logger, null_logger

from EuCAP2024.umbms.loadsave import load_pickle, save_pickle

from EuCAP2024.umbms.recon.extras import get_pix_ts, get_pix_ds

from EuCAP2024.umbms.tdelay.propspeed import estimate_speed

from EuCAP2024.umbms.hardware.antenna import apply_ant_t_delay, to_phase_center

from EuCAP2024.umbms.plot.imgs import plot_img

from skimage.metrics import structural_similarity as ssim
import numpy as np



###############################################################################

# Directory to load UM-BMID Gen-3 data
__D_DIR = os.path.join(get_proj_path(),
                       'data/gen-two/clean/')

# Output dir for saving
__O_DIR = os.path.join(get_proj_path(), "output")
verify_path(__O_DIR)

# The frequency parameters from the scan
__INI_F = 1e9
__FIN_F = 8e9
__N_FS = 1001
__SCAN_FS = np.linspace(__INI_F, __FIN_F, __N_FS)
__SCAN_FS_short = __SCAN_FS[139::3] #I didn't take it from 2GHz exactly during predictions.


__M_SIZE = 150  # Number of pixels along 1-dimension for reconstruction
__ROI_RAD = 8  # ROI radius, in [cm]

__C = 2.99792458e8

# The approximate radius of each adipose phantom in our dataset,
# defined as the median radii
__ADI_RADS = {
    'A1': 3.87,
    'A2': 5.00,
    'A3': 5.66,
    'A11': 4.48,
    'A12': 4.57,
    'A13': 4.84,
    'A14': 5.19,
    'A15': 5.53,
    'A16': 5.74,
}



if __name__ == "__main__":

    logger = get_script_logger("notebook_run")

    # Flags for doing DAS / DMAS / ORR reconstructions
    do_das = True
    do_dmas = False
    do_orr = False


    # If True, ORR will use GPU for computation
    orr_use_gpu = False

    s11_orig = np.array(load_pickle('data_wos_test.pickle'))
    md = (load_pickle('metadata_wos_test.pickle'))

    s11_pred = np.array(load_pickle('predicted_test_v2.pickle'))
    
    recon_fs = __SCAN_FS_short
    #print('norm',recon_fs)


    s11_orig = s11_orig[:, 139::3, :]
    s11_pred = s11_pred[:,:,:,0]

    print(s11_orig.shape, s11_pred.shape)

    # Lists to store metrics
    all_mse = []
    all_ssim = []
    all_mse_norm =[]
    all_ssim_norm =[]
    

    n_expts = len(md)  # Find number of expts

    # Get the unique ID of each experiment / scan
    expt_ids = [mm['id'] for mm in md]

    # Get the unique IDs of the adipose-only and adipose-fibroglandular
    # (healthy) reference scans for each experiment/scan
    adi_ref_ids = [mm['adi_ref_id'] for mm in md]
    fib_ref_ids = [mm['fib_ref_id'] for mm in md]

    # List of physics enhanced modelling factors to use
    phys_enhancements = [
        # 'beam',
        # 'spherical',
        # 'gain',
        # 'sec_scat',
        # 'complex_k',
        # 'speed',
    ]

    if len(phys_enhancements) >= 1:  # If using some enhancements...

        # The output dir, where the reconstructions will be stored
        out_dir = os.path.join(__O_DIR, '-'.join(phys_enhancements)
                               + "-median/")
        verify_path(out_dir)

    else:  # If not using enhancements...

        out_dir = os.path.join(__O_DIR, "base-median-adi-rads/")
        verify_path(out_dir)

    # Define output dir for adipose and healthy references
    adi_o_dir = os.path.join(out_dir, 'adi/')
    verify_path(adi_o_dir)
    healthy_o_dir = os.path.join(out_dir, "healthy/")
    verify_path(healthy_o_dir)

    # Init lists for storing reconstructions using adipose reference
    all_das_adi_original = []
    all_das_adi_pred = []
    all_dmas_adi = []
    all_orr_adi = []

    # Init lists for storing reconstructions using healthy reference
    all_das_fib = []
    all_dmas_fib = []
    all_orr_fib = []

    eps_0 = 8.85e-12  # Vacuum permittivity

    step_size = 0.3  # Select the step size

    # Init lists for storing metadata
    adi_md_to_save = []
    fib_md_to_save = []

    for ii in range(n_expts):  # For each scan / experiment

        logger.info('Scan [%3d / %3d]...' % (ii + 1, n_expts))

        # Get the frequency domain data and metadata of this experiment
        tar_fd_original = s11_orig[ii, :, :]
        tar_fd_pred = s11_pred[ii, :, :]
        tar_md = md[ii]

        # If the scan had a fibroglandular shell (indicating it was of
        # a complete tumour-containing or healthy phantom)
        if 'F' in tar_md['phant_id']:

            logger.info("\tDoing adipose-only reference reconstructions...")

            adi_md_to_save.append(tar_md)  # Store scan metadata

            # Get metadata for plotting
            scan_rad = tar_md['ant_rad']
            tum_x = tar_md['tum_x']
            tum_y = tar_md['tum_y']
            tum_rad = (tar_md['tum_rad'])
            adi_rad = __ADI_RADS[tar_md['phant_id'].split('F')[0]]

            # Correct for the antenna time delay
            # NOTE: Only the new antenna was used in UM-BMID Gen-3
            ant_rad = to_phase_center(meas_rho=scan_rad)

            # Estimate the propagation speed in the imaging domain
            speed = estimate_speed(adi_rad=adi_rad,
                                   ant_rho=ant_rad,
                                   )


            # Get the one-way propagation times for each pixel,
            # for each antenna position
            pix_ts = get_pix_ts(ant_rho=ant_rad,
                                m_size=__M_SIZE,
                                roi_rad=__ROI_RAD,
                                speed=speed
                                )


            # Apply the antenna time delay
            pix_ts = apply_ant_t_delay(pix_ts=pix_ts)
            
            # Get the phase factor for efficient computation
            phase_fac = np.exp(-1j
                               * 2 * np.pi * recon_fs[:, None, None, None]
                               * pix_ts[None, :, :, :]
                               )



            # Subtract reference
            adi_cal_cropped_original = tar_fd_original
            adi_cal_cropped_pred = tar_fd_pred

            # If the scan does include a tumour
            if ~np.isnan(tar_md['tum_rad']):

                # Set a str for plotting
                plt_str = "%.1f cm tum in " \
                          "Class %d %s, ID: %d" % (tar_md['tum_rad'],
                                                   tar_md['birads'],
                                                   tar_md['phant_id'],
                                                   tar_md['id'])

            else:  # If the scan does NOT include a tumour
                plt_str = "Class %d %s, ID: %d" % (tar_md['birads'],
                                                   tar_md['phant_id'],
                                                   tar_md['id'])

            if do_das:  # If doing DAS reconstructions

                das_t0 = time.time()
                logger.info("\tStarting DAS recon...")

                # Reconstruct a DAS image
                das_adi_recon_original = np.sum(
                    adi_cal_cropped_original[:, :, None, None]
                    * phase_fac ** (-2),
                    axis=(0, 1)
                )

                das_adi_recon_pred = np.sum(
                    adi_cal_cropped_pred[:, :, None, None]
                    * phase_fac ** (-2),
                    axis=(0, 1)
                )

                das_orig_abs = np.abs(das_adi_recon_original)
                das_pred_abs = np.abs(das_adi_recon_pred)

                das_orig_abs_m = das_orig_abs / np.max(np.abs(das_adi_recon_original))
                das_pred_abs_m = das_pred_abs / np.max(np.abs(das_adi_recon_pred))

                mse_val = np.mean((das_orig_abs - das_pred_abs) ** 2)
                ssim_val = ssim(das_orig_abs, das_pred_abs, data_range=1.0)

                mse_val_norm = np.mean((das_orig_abs_m - das_pred_abs_m) ** 2)
                ssim_val_norm = ssim(das_orig_abs_m, das_pred_abs_m, data_range=das_orig_abs_m.max() - das_orig_abs_m.min())

                all_mse.append(mse_val)
                all_ssim.append(ssim_val)

                all_mse_norm.append(mse_val_norm)
                all_ssim_norm.append(ssim_val_norm)
                
                #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                
                # Store DAS image
                all_das_adi_original.append(das_adi_recon_original * np.ones_like(das_adi_recon_original))
                all_das_adi_pred.append(das_adi_recon_pred * np.ones_like(das_adi_recon_pred))

                # Plot the DAS reconstruction
                plot_img(img=np.abs(das_adi_recon_original),
                         tar_xs=[tum_x],
                         tar_ys=[tum_y],
                         tar_rads=[tum_rad],
                         roi_rho=__ROI_RAD,
                         title='%s\nOriginal' % plt_str ,
                         save_fig=True,
                         save_str=os.path.join(adi_o_dir,
                                               'idx_%d_original_das.png'
                                               % ii),
                         save_close=True
                         )
                print(adi_o_dir, 'idx_%d_original_das.png'% ii)

                plot_img(img=np.abs(das_adi_recon_pred),
                         tar_xs=[tum_x],
                         tar_ys=[tum_y],
                         tar_rads=[tum_rad],
                         roi_rho=__ROI_RAD,
                         title='%s\nPredicted' % plt_str,
                         save_fig=True,
                         save_str=os.path.join(adi_o_dir,
                                               'idx_%d_predicted_das.png'
                                               % ii),
                         save_close=True
                         )
                logger.info("\t\tFinished DAS in %.1f sec."
                            % (time.time() - das_t0))
    avg_mse = np.mean(all_mse_norm)
    avg_ssim = np.mean(all_ssim_norm)

    print("MSE:", all_mse)
    print("SSIM:", all_ssim)

    print("MSE normalized:", all_mse_norm)
    print("SSIM normalized:", all_ssim_norm)

    print("MSE average:", avg_mse)
    print("SSIM average:", avg_ssim)  

[2025-09-28 11:28:41,985][INFO][__init__.py:132]:	UM-BMS Logger initialized in host [Manuel].
(51, 288, 72) (51, 288, 72)
[2025-09-28 11:28:43,111][INFO][3012232027.py:161]:	Scan [  1 /  51]...
[2025-09-28 11:28:43,114][INFO][3012232027.py:172]:		Doing adipose-only reference reconstructions...
[2025-09-28 11:29:23,358][INFO][3012232027.py:235]:		Starting DAS recon...
c:\Users\andre\OneDrive\Documentos\Gabi\report\DAS (2)\DAS (2)\DAS\EuCAP2024\output\base-median-adi-rads/adi/ idx_0_original_das.png
[2025-09-28 11:32:21,227][INFO][3012232027.py:301]:			Finished DAS in 177.9 sec.
[2025-09-28 11:32:21,229][INFO][3012232027.py:161]:	Scan [  2 /  51]...
[2025-09-28 11:32:21,232][INFO][3012232027.py:172]:		Doing adipose-only reference reconstructions...
[2025-09-28 11:32:48,565][INFO][3012232027.py:235]:		Starting DAS recon...
c:\Users\andre\OneDrive\Documentos\Gabi\report\DAS (2)\DAS (2)\DAS\EuCAP2024\output\base-median-adi-rads/adi/ idx_1_original_das.png
[2025-09-28 11:35:41,289][INFO][301